In [4]:
import easyocr
import cv2
import os
import numpy as np
from PIL import Image
import pdf2image
import pickle
import torch
import pandas as pd
reader = easyocr.Reader(['en'])

In [5]:
with open('400_leases_list.pkl', 'rb') as f:
    files_with_400 = pickle.load(f)

df = pd.DataFrame(columns = ['id','source_sentence','source_file','sentence_split'])